### Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from scipy.stats import qmc

### Illustrate sampling method 
Sampling a normal distribtuion using random vs. stratified sampling

In [ ]:
# Experiment parameters, feel free to play around with these!
num_samples = 200
seed = 123

rng = np.random.default_rng(seed)

# Random normal sample
normal_values = rng.normal(loc=0.0, scale=1.0, size=num_samples)

# LHS quantiles in [0, 1], mapped to N(0,1) via inverse CDF (PPF)
sampler_1d = qmc.LatinHypercube(d=1, seed=seed)
quantiles = sampler_1d.random(n=num_samples).ravel()
lhs_values = sp.stats.norm.ppf(quantiles)

# Plot
fig, ax = plt.subplots(figsize=(6, 4))
ax.hist(normal_values, bins=30, alpha=0.5, density=True, label="Random N(0,1)")
ax.hist(
    lhs_values,
    bins=30,
    alpha=0.5,
    density=True,
    label="LHS via PPF",
    color="tab:orange",
)

x = np.linspace(-4, 4, 200)
ax.plot(x, sp.stats.norm.pdf(x), "k--", label="True PDF")

ax.set_title("Random vs stratified (LHS) sampling of N(0,1)")
ax.set_xlabel("Value")
ax.set_ylabel("Density")
ax.legend()
plt.tight_layout()
plt.show()

### Compare the covrege of stratified sampling vs. random sampling

In [ ]:
def coverage_comparison(
    number_of_parameters: int = 30,
    realizations: int = 100,
    seed: int = 123,
) -> None:
    min_dist_random = np.empty(realizations)
    min_dist_lhs = np.empty(realizations)
    l2star_random = np.empty(realizations)
    l2star_lhs = np.empty(realizations)

    for i in range(realizations):
        seed_i = seed + i
        # Random uniform samples in [0, 1]^d, with d = number of parameters
        rng_random = np.random.default_rng(seed_i)
        samples_random = rng_random.random(size=(number_of_parameters, 1))
        min_dist_random[i] = qmc.geometric_discrepancy(samples_random, method="mindist")
        l2star_random[i] = qmc.discrepancy(samples_random, method="L2-star")

        # Latin Hypercube samples in [0, 1]^d, with d = number of parameters
        sampler = qmc.LatinHypercube(d=1, seed=seed_i)
        samples_lhs = sampler.random(number_of_parameters)
        min_dist_lhs[i] = qmc.geometric_discrepancy(samples_lhs, method="mindist")
        l2star_lhs[i] = qmc.discrepancy(samples_lhs, method="L2-star")

    # Plot histograms of the metrics
    _fig, axs = plt.subplots(1, 2, figsize=(10, 3))
    binning = "fd"  # Freedman–Diaconis rule

    axs[0].hist(min_dist_random, bins=binning, alpha=0.5, label="Random")
    axs[0].hist(min_dist_lhs, bins=binning, alpha=0.5, label="LHS")
    axs[0].set_title(f"Min pairwise distance in {number_of_parameters}D")
    axs[0].set_xlabel("min(||x_i - x_j||)")
    axs[0].set_ylabel("Count")
    axs[0].legend()

    axs[1].hist(l2star_random, bins=binning, alpha=0.5, label="Random")
    axs[1].hist(l2star_lhs, bins=binning, alpha=0.5, label="LHS")
    axs[1].set_title(f"L2-star discrepancy in {number_of_parameters}D")
    axs[1].set_xlabel("Discrepancy")
    axs[1].set_ylabel("Count")
    axs[1].legend()

    plt.tight_layout()
    plt.show()

In [ ]:
coverage_comparison(number_of_parameters=6, realizations=100)